# Setup

In [1]:
# Basic Imports
from math import sin, cos, sqrt, atan2, radians

# Data Wrangling
import pandas as pd
import numpy as np

# Data Visualization
import folium

# Folium Plugins and Features
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

# Loading Data

In [2]:
spacex_df = (
    pd
    .read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv")
    .loc[:, ["Launch Site", "Lat", "Long", "class"]]
)

launch_sites_df = (
    spacex_df
    .loc[:, ["Launch Site", "Lat", "Long"]]
    .groupby("Launch Site", as_index=False).first()
)

In [9]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


# Creating Maps

Task 1: Mark all launch sites on a map

In [3]:
#  NASA Johnson Space Center's
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color="#0000FF", fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#0000FF;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

# Creating the same for Launch Sites
for launch_site in launch_sites_df["Launch Site"]:
    location = launch_sites_df.loc[launch_sites_df["Launch Site"].eq(launch_site), ["Lat", "Long"]].squeeze().tolist()
    # Circle
    circle = folium.Circle(
        location=location,
        radius=1000,
        color="#0000FF",
        fill=True
    ).add_child(folium.Popup(launch_site)) 
    # Marker
    marker = folium.map.Marker(
        location=location,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#0000FF;"><b>%s</b></div>' % f'{launch_site}',
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)

display(site_map)

Task 2: Mark the success/failed launches for each site on the map

In [4]:
# Creating cluster of markers
marker_cluster = MarkerCluster()

# Defining the color for each type of launch
spacex_df["marker_color"] = np.where(spacex_df["class"].eq(1), "#007500", "#A30000")

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for _, row in spacex_df.iterrows():
    location = row[["Lat", "Long"]].squeeze().tolist()
    marker = folium.map.Marker(
        location=location,
        icon=folium.Icon(color="white", icon_color=row["marker_color"])
        )
    marker_cluster.add_child(marker)

display(site_map)

Task 3: Calculate the distances between a launch site to its proximities

In [5]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position="topright",
    separator=" Long: ",
    empty_string="NaN",
    lng_first=False,
    num_digits=20,
    prefix="Lat:",
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
display(site_map)

In [6]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [7]:
launch_site_coordinates = spacex_df.loc[spacex_df["Launch Site"].eq("CCAFS SLC-40"), ["Lat", "Long"]].iloc[0].squeeze().tolist()
closest_coastline_coordinates = (28.56367, -80.56798)
closest_highway_coordinates = (28.56335, -80.57085)
closest_railroad_coordinates = (28.57206, -80.58525)
closest_city_coordinates = (28.10473, -80.64531)

points_of_interest = [
    closest_coastline_coordinates,
    closest_highway_coordinates,
    closest_railroad_coordinates,
    closest_city_coordinates,
]

In [8]:
for point in points_of_interest:
    distance_marker = folium.Marker(
       point,
       icon=DivIcon(
           icon_size=(20,20),
           icon_anchor=(0,0),
           html='<div style="font-size: 12; color:#0000FF;"><b>%s</b></div>' % "{:10.2f} KM".format(
               calculate_distance(*launch_site_coordinates, *point)
               ),
           )
       )
    site_map.add_child(distance_marker)

    lines=folium.PolyLine(locations=(launch_site_coordinates, point), weight=1)
    site_map.add_child(lines)

display(site_map)